In [44]:
# Based on docs:
# https://docs.lightly.ai/self-supervised-learning/tutorials/package/tutorial_simclr_clothing.html

# Also, see:
# https://github.com/giakoumoglou/classification/blob/main/notebooks/main_simclr.ipynb
# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html

## Imports

In [61]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize

from lightly.data import LightlyDataset
from lightly.transforms import SimCLRTransform, utils

## Configuration

We set some configuration parameters for our experiment.
Feel free to change them and analyze the effect.

The default configuration with a batch size of 256 and input resolution of 128
requires 6GB of GPU memory.



In [46]:
num_workers = 8
batch_size = 256
seed = 1
max_epochs = 20
input_size = 128
num_ftrs = 32

Let's set the seed for our experiments



In [47]:
pl.seed_everything(seed)

Seed set to 1


1

In [ ]:
# Create the directory if it doesn't exist
os.makedirs("./datasets/mnist", exist_ok=True)
os.makedirs("./datasets/mnist/train", exist_ok=True)
os.makedirs("./datasets/mnist/test", exist_ok=True)

In [ ]:
# Download the MNIST dataset
train_dataset = datasets.MNIST(root="./datasets/mnist", train=True, download=True)
test_dataset = datasets.MNIST(root="./datasets/mnist", train=False, download=True)

In [ ]:
# Save the images to the directory
for i, (image, label) in enumerate(train_dataset):
    image.save(f"./datasets/mnist/train/{i}.png")

for i, (image, label) in enumerate(test_dataset):
    image.save(f"./datasets/mnist/test/{i}.png")

In [ ]:
# Remove original data to prevent errors
shutil.rmtree("./datasets/mnist/MNIST")

In [48]:
path_to_data = r"./datasets/mnist"

## Setup data augmentations and loaders

The images from the dataset have been taken from above when the clothing was
on a table, bed or floor. Therefore, we can make use of additional augmentations
such as vertical flip or random rotation (90 degrees).
By adding these augmentations we learn our model invariance regarding the
orientation of the clothing piece. E.g. we don't care if a shirt is upside down
but more about the strcture which make it a shirt.

You can learn more about the different augmentations and learned invariances
here: `lightly-advanced`.



In [49]:
transform = SimCLRTransform(input_size=input_size, vf_prob=0.5, rr_prob=0.5)

# We create a torchvision transformation for embedding the dataset after
# training
test_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((input_size, input_size)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=utils.IMAGENET_NORMALIZE["mean"],
            std=utils.IMAGENET_NORMALIZE["std"],
        ),
    ]
)

dataset_train_simclr = LightlyDataset(input_dir=path_to_data, transform=transform)

dataset_test = LightlyDataset(input_dir=path_to_data, transform=test_transform)

dataloader_train_simclr = torch.utils.data.DataLoader(
    dataset_train_simclr,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
)

## Create the SimCLR Model
Now we create the SimCLR model. We implement it as a PyTorch Lightning Module
and use a ResNet-18 backbone from Torchvision. Lightly provides implementations
of the SimCLR projection head and loss function in the `SimCLRProjectionHead`
and `NTXentLoss` classes. We can simply import them and combine the building
blocks in the module.



In [50]:
from lightly.loss import NTXentLoss
from lightly.models.modules.heads import SimCLRProjectionHead


class SimCLRModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # create a ResNet backbone and remove the classification head
        resnet = torchvision.models.resnet18()
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])

        hidden_dim = resnet.fc.in_features
        self.projection_head = SimCLRProjectionHead(hidden_dim, hidden_dim, 128)

        self.criterion = NTXentLoss()

    def forward(self, x):
        h = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(h)
        return z

    def training_step(self, batch, batch_idx):
        (x0, x1), _, _ = batch
        z0 = self.forward(x0)
        z1 = self.forward(x1)
        loss = self.criterion(z0, z1)
        self.log("train_loss_ssl", loss)
        return loss

    def configure_optimizers(self):
        optim = torch.optim.SGD(
            self.parameters(), lr=6e-2, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, max_epochs)
        return [optim], [scheduler]

Train the module using the PyTorch Lightning Trainer on a single GPU.



In [51]:
# Uncomment for training (may take some time)

# model = SimCLRModel()
# trainer = pl.Trainer(max_epochs=max_epochs, devices=1, accelerator="gpu")
# trainer.fit(model, dataloader_train_simclr)

# TODO: add early stopping, as the model doesn't improve further after around 10 epochs

Next we create a helper function to generate embeddings
from our test images using the model we just trained.
Note that only the backbone is needed to generate embeddings,
the projection head is only required for the training.
Make sure to put the model into eval mode for this part!



In [52]:
def generate_embeddings(model, dataloader):
    """Generates representations for all images in the dataloader with
    the given model
    """

    embeddings = []
    filenames = []
    with torch.no_grad():
        for img, _, fnames in dataloader:
            img = img.to(model.device)
            emb = model.backbone(img).flatten(start_dim=1)
            embeddings.append(emb)
            filenames.extend(fnames)

    embeddings = torch.cat(embeddings, 0)
    embeddings = normalize(embeddings.to("cpu"))
    return embeddings, filenames

In [53]:
# For eval only
model = SimCLRModel.load_from_checkpoint(
    "./lightning_logs/version_4/checkpoints/epoch=14-step=4095.ckpt"
)

In [54]:
model.eval()
embeddings, filenames = generate_embeddings(model, dataloader_test)

## Visualize Nearest Neighbors
Let's look at the trained embedding and visualize the nearest neighbors for
a few random samples.

We create some helper functions to simplify the work



In [62]:
def get_distances_between_centroids(
    embeddings: np.ndarray = None, n_clusters: int = 10
) -> np.ndarray:
    """Calculate the clusters and distances between their centroids.

    Args:
        embeddings (np.ndarray, optional): Input embeddings. Defaults to None.
        n_clusters (int, optional): Should be 10 for MNIST, but generally is data-dependent. Defaults to 10.

    Returns:
        np.ndarray: matrix of distances between centroids, with rank n_clusters x n_clusters
    """
    kmeans = KMeans(n_clusters=n_clusters)
    labels = kmeans.fit_predict(embeddings)
    centroids = kmeans.cluster_centers_

    print(f"Cluster centroids dimensions: {centroids.shape}")
    print(f"Cluster centroids:\n {centroids}")
    print(f"Cluster sizes: {np.bincount(labels)}")

    distances = cdist(centroids, centroids)
    return distances

In [63]:
distances = get_distances_between_centroids(embeddings)

Cluster centroids dimensions: (10, 512)
Cluster centroids:
 [[0.01356608 0.03640075 0.03651705 ... 0.02792558 0.04370748 0.04605724]
 [0.04092902 0.04653256 0.04202884 ... 0.05929984 0.02878389 0.0153493 ]
 [0.03477266 0.07670884 0.03996711 ... 0.04230779 0.03355276 0.05636859]
 ...
 [0.0236144  0.01617302 0.01519194 ... 0.05519713 0.05978007 0.00975647]
 [0.03657048 0.05329192 0.0352304  ... 0.02297343 0.01952295 0.04115388]
 [0.01952146 0.0458184  0.03055629 ... 0.03901875 0.0276754  0.01693676]]
Cluster sizes: [9886 6703 8586 7351 4235 6987 6513 5033 5463 9243]


In [67]:
print(type(distances))
print(distances.shape)
print(np.linalg.matrix_rank(distances))
print(distances)

<class 'numpy.ndarray'>
(10, 10)
10
[[0.         0.60001089 0.44586291 0.50427062 0.73007753 0.59723433
  0.510906   0.66784589 0.60123537 0.52751229]
 [0.60001089 0.         0.56162077 0.47035942 0.4782806  0.50896548
  0.71032653 0.57670742 0.54109041 0.43905751]
 [0.44586291 0.56162077 0.         0.4336027  0.70515167 0.57630494
  0.5113117  0.69043428 0.46001387 0.6326839 ]
 [0.50427062 0.47035942 0.4336027  0.         0.68746722 0.48827978
  0.52401121 0.67088874 0.50796434 0.43496134]
 [0.73007753 0.4782806  0.70515167 0.68746722 0.         0.66934224
  0.83559682 0.46668641 0.69847081 0.66702631]
 [0.59723433 0.50896548 0.57630494 0.48827978 0.66934224 0.
  0.77826917 0.57589671 0.71926609 0.59761182]
 [0.510906   0.71032653 0.5113117  0.52401121 0.83559682 0.77826917
  0.         0.80987732 0.67091265 0.56816359]
 [0.66784589 0.57670742 0.69043428 0.67088874 0.46668641 0.57589671
  0.80987732 0.         0.70984386 0.70857458]
 [0.60123537 0.54109041 0.46001387 0.50796434 0.6984

In [57]:
def get_image_as_np_array(filename: str):
    """Returns an image as an numpy array"""
    img = Image.open(filename)
    return np.asarray(img)

In [ ]:
def plot_knn_examples(embeddings, filenames, n_neighbors=3, num_examples=6):
    """Plots multiple rows of random images with their nearest neighbors"""
    # lets look at the nearest neighbors for some samples
    # we use the sklearn library
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(embeddings)
    distances, indices = nbrs.kneighbors(embeddings)

    # get 5 random samples
    samples_idx = np.random.choice(len(indices), size=num_examples, replace=False)

    # loop through our randomly picked samples
    for idx in samples_idx:
        fig = plt.figure()
        # loop through their nearest neighbors
        for plot_x_offset, neighbor_idx in enumerate(indices[idx]):
            # add the subplot
            ax = fig.add_subplot(1, len(indices[idx]), plot_x_offset + 1)
            # get the correponding filename for the current index
            fname = os.path.join(path_to_data, filenames[neighbor_idx])
            # plot the image
            plt.imshow(get_image_as_np_array(fname))
            # set the title to the distance of the neighbor
            ax.set_title(f"d={distances[idx][plot_x_offset]:.3f}")
            # let's disable the axis
            plt.axis("off")

In [ ]:
plot_knn_examples(embeddings, filenames, n_neighbors=10, num_examples=10)

In [ ]:
plot_knn_examples(embeddings, filenames)

## Clusters
Let's look at the trained embedding and visualize clusters in latent space.

In [ ]:
def plot_clusters(
    embeddings: np.ndarray = None,
    n_clusters: int = 10,
    proportion_of_points_to_plot: float = 0.001,
) -> None:
    """Plots multiple rows of random images with their cluster centroids"""

    print(f"Working on embeddings of shape {embeddings.shape}")

    reduced_embeddings = PCA(n_components=2).fit_transform(embeddings)

    kmeans = KMeans(n_clusters=n_clusters)
    labels = kmeans.fit_predict(reduced_embeddings)
    centroids = kmeans.cluster_centers_

    print(f"Cluster centroids:\n {centroids}")
    print(f"Cluster sizes: {np.bincount(labels)}")

    # Sample proportion_of_points_to_plot of the data to plot for readability
    sampled_indices = np.random.choice(
        reduced_embeddings.shape[0],
        int(len(reduced_embeddings) * proportion_of_points_to_plot),
        replace=False,
    )
    sampled_embeddings = reduced_embeddings[sampled_indices]
    sampled_labels = labels[sampled_indices]

    print(f"Plotting {len(sampled_embeddings)} points out of {len(reduced_embeddings)}")

    plt.scatter(sampled_embeddings[:, 0], sampled_embeddings[:, 1], c=sampled_labels)
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.title("Cluster Plot")
    plt.show()

In [ ]:
plot_clusters(embeddings, proportion_of_points_to_plot=0.01)